In [10]:
import requests
from selenium import webdriver
import pandas as pd
import matplotlib.pyplot as plt
import re
import pickle
import nltk
from nltk.corpus import stopwords
import string
from collections import Counter
from itertools import chain

In [11]:
from ipynb.fs.full.functions import next_page, clean_date, sort_df, find_ngrams, bigramas, freq_bigram  

In [12]:
url = 'https://repository.urosario.edu.co/handle/10336/622/recent-submissions'

In [15]:
options = webdriver.ChromeOptions()
#Le agregamos argumentos
options.add_argument('--incognito')
#Esto va a abrir un navegador vacío que dice que un software automatizado controla 
driver = webdriver.Chrome(executable_path='../chromedriver', options=options)

driver.get(url)

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [5]:
def obtener_tesis(driver):
    
    #Creamos una lista vacía para que contenga todas las tesis 
    tesis_total = []
    # Obtenemos los impares
    tesis_total.append(driver.find_elements_by_xpath('//li[@class="ds-artifact-item odd"]'))
    # Obtenemos los pares
    tesis_total.append(driver.find_elements_by_xpath('//li[@class="ds-artifact-item even"]'))
    # Transformamos la lista de listas en una sola lista 
    tesis_total =[t for l in tesis_total for t in l]
    return tesis_total

In [6]:
info = []
def obtener_info(driver):
    tesis_totales = obtener_tesis(driver)
    print('iniciando scraping...')
    for tesis in tesis_totales:
        # Obtenemos fecha
        fecha = tesis.find_element_by_xpath('.//span[@class="item-properties"]').text
        # Obtenemos título
        titulo = tesis.find_element_by_xpath('.//h4[@class="artifact-title"]').text
        info_dict={'Fecha': fecha, 'Título': titulo}
        info.append(info_dict)
    return info

In [8]:
def main (driver):
    for x in range (1,40):
        obtener_info(driver)
        next_page(driver)
    return info

In [9]:
main(driver)

iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...


[{'Fecha': ' Trabajo de grado. 2004',
  'Título': 'El Secretariado Nacional de Pastoral Social / Caritas Colombiana: un gestor de Cooperación Internacional al Desarrollo en Colombia'},
 {'Fecha': ' Trabajo de grado. 2005',
  'Título': 'Impacto de la ley 715 de 2001 en el sector educativo del distrito de Santa Marta durante el periodo 2002,2003 y 2004'},
 {'Fecha': ' Trabajo de grado. 2005',
  'Título': 'Análisis de la efectividad de los consejos comunales de gobierno del presidente Álvaro Uribe Vélez, en materia de logros y participación durante sus dos primeros años de gobierno'},
 {'Fecha': ' Trabajo de grado. 2005',
  'Título': 'Estado y telecomunicaciones: la privatización del sector mediante la adopción de la telefonía celular'},
 {'Fecha': ' Trabajo de grado. 2005',
  'Título': 'Representación mediática del Estado Comunitario: aproximación desde el periódico El Tiempo entre el 8 julio y el 7 de octubre de 2002'},
 {'Fecha': ' Trabajo de grado. 2004',
  'Título': 'Las áreas proteg

In [10]:
driver.close()

Convertimos el archivo info en un DataFrame

In [11]:
df = pd.DataFrame(info)

Cambiamos los valores dentro de fecha a años

In [13]:
regex = r'(\d{1,4})'
clean_date(df, regex)

,Fecha,Título
0,2004,El Secretariado Nacional de Pastoral Social / ...
1,2005,Impacto de la ley 715 de 2001 en el sector edu...
2,2005,Análisis de la efectividad de los consejos com...
3,2005,Estado y telecomunicaciones: la privatización ...
4,2005,Representación mediática del Estado Comunitari...
...,...,...
770,2008,La formación de actividades ilegales en la ciu...
771,2009,Las limitantes de construcción de ciudadanía i...
772,2009,Configuración del concepto de Estado Vasco a p...
773,2009,Análisis de la incidencia de la Sentencia T-02...


Volvemos el valor en fecha un integrer

In [14]:
df['Fecha'] = df['Fecha'].astype(int)

Limpiamos los títulos de las tesis de signos de puntuación y signos speciales. Además se tokenizan los títulos y se ponen todas las palabras en minúscula

In [1]:
def clean_titulo(df):
    """Esta función recibe un Data Frame y a la columna con """
    # Definimos los signos de puntiación en una variable
    punt_espa = string.punctuation + "–¡¿”“•\r"
    
    # Definimos función para quitar los signos de puntiación 
    def sin_punt(x):
        x = ''.join([i for i in x if i not in (punt_espa)])
        return x
    
    # Aplicamos la función para quitar los signos
    df['Título cleaned'] = df['Título'].apply(sin_punt)
    
    # Definimos las stopwords
    stopwords_esp = stopwords.words('spanish')
    stopwords_esp = stopwords_esp + ['estudio', 'caso', 'análisis', 'bogotá', 'periodo', 'incidencia', 'colombia', 'político', 'hacia', 'impacto', 'the', 'and', 'aproximación', 'influencia', 'factores', 'medio', 'interés', 'programa', 'influencia']
    
    
    # Ponemos las palabras todas en minúscula y las separamos
    df['Título cleaned'] = df['Título cleaned'].str.lower().str.split()
    
    # Eliminamos las stopwords
    df['Título cleaned'] = df['Título cleaned'].apply(lambda x: [item for item in x if item not in stopwords_esp])
    
    
    return df

In [2]:
clean_titulo(df)

NameError: name 'df' is not defined

Extraemos los bigramas de los títulos

In [18]:
bigramas(df)

,Fecha,Título,Título cleaned,bigrams
0,2004,El Secretariado Nacional de Pastoral Social / ...,"[secretariado, nacional, pastoral, social, car...","[(secretariado, nacional), (nacional, pastoral..."
1,2005,Impacto de la ley 715 de 2001 en el sector edu...,"[impacto, ley, 715, 2001, sector, educativo, d...","[(impacto, ley), (ley, 715), (715, 2001), (200..."
2,2005,Análisis de la efectividad de los consejos com...,"[efectividad, consejos, comunales, gobierno, p...","[(efectividad, consejos), (consejos, comunales..."
3,2005,Estado y telecomunicaciones: la privatización ...,"[telecomunicaciones, privatización, sector, me...","[(telecomunicaciones, privatización), (privati..."
4,2005,Representación mediática del Estado Comunitari...,"[representación, mediática, comunitario, aprox...","[(representación, mediática), (mediática, comu..."
...,...,...,...,...
770,2008,La formación de actividades ilegales en la ciu...,"[formación, actividades, ilegales, ciudad, car...","[(formación, actividades), (actividades, ilega..."
771,2009,Las limitantes de construcción de ciudadanía i...,"[limitantes, construcción, ciudadanía, integra...","[(limitantes, construcción), (construcción, ci..."
772,2009,Configuración del concepto de Estado Vasco a p...,"[configuración, concepto, vasco, partir, relac...","[(configuración, concepto), (concepto, vasco),..."
773,2009,Análisis de la incidencia de la Sentencia T-02...,"[sentencia, t0252004, corte, constitucional, p...","[(sentencia, t0252004), (t0252004, corte), (co..."


Guardamos el data frame en un archivo pickle para después trabajar con él

In [20]:
df.to_pickle('repo_UR.pkl')